In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [ ]:
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.mnist.load_data()

In [ ]:
# 데이터 확인

print( X_train.shape )
X_train[0, :10, :10]

(60000, 28, 28)


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36],
       [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253],
       [  0,   0,   0,   0,   0,   0,   0,  18, 219, 253],
       [  0,   0,   0,   0,   0,   0,   0,   0,  80, 156]], dtype=uint8)

In [ ]:
# 데이터 변환 0 ~ 255 -> 0 ~ 1
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

# X_train_scaled[0, 10:20, 10:20]
X_train_scaled.min(), X_train_scaled.max()

(0.0, 1.0)

In [ ]:
# 모델 구조 설계

model = tf_keras.models.Sequential()
model.add(tf_keras.layers.Input(shape=(28, 28, 1)))
model.add(tf_keras.layers.Conv2D(filters=32,
                                 kernel_size=3, # 3 x 3 크기의 필터 사용
                                 padding='same', # zero padding을 적용해서 입력 이미지의 크기와 출력 피처맵의 크기를 같게 설정
                                 activation='relu'))
model.add(tf_keras.layers.MaxPool2D(pool_size=2))
model.add(tf_keras.layers.Conv2D(filters=64,
                                 kernel_size=3,
                                 padding='same',
                                 activation='relu'))
model.add(tf_keras.layers.MaxPool2D(pool_size=2))

model.add(tf_keras.layers.Flatten())

model.add(tf_keras.layers.Dense(units=128, activation='relu'))
model.add(tf_keras.layers.Dense(units=10, activation='softmax'))

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 421,642 (1.61 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 모델 학습 설계

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# 모델 학습 (훈련)

fit_history = model.fit(X_train_scaled, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 65s 85ms/step - accuracy: 0.8814 - loss: 0.3971 - val_accuracy: 0.9818 - val_loss: 0.0606
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 84ms/step - accuracy: 0.9827 - loss: 0.0553 - val_accuracy: 0.9853 - val_loss: 0.0495
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 85ms/step - accuracy: 0.9892 - loss: 0.0328 - val_accuracy: 0.9874 - val_loss: 0.0453
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 84ms/step - accuracy: 0.9927 - loss: 0.0237 - val_accuracy: 0.9891 - val_loss: 0.0376
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 85ms/step - accuracy: 0.9934 - loss: 0.0177 - val_accuracy: 0.9876 - val_loss: 0.0433
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 85ms/step - accuracy: 0.9962 - loss: 0.0121 - val_accuracy: 0.9887 - val_loss: 0.0461
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 62s 83ms/step - accuracy: 0.9971 - loss: 0.0086 - val_accuracy: 0.9887 - val_loss: 0.0389
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 63s 83ms/step - accuracy: 0.9972 - loss: 0.0084 - 

In [ ]:
model.evaluate(X_test_scaled, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9898 - loss: 0.0541


[0.04251857474446297, 0.9908999800682068]